#سیستم توصیه گر پیشنهاد دستور پخت غذا

#download crawled dataset form google drive

In [3]:
!mkdir /content/images
!wget --header 'Host: drive.usercontent.google.com' --user-agent 'Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:123.0) Gecko/20100101 Firefox/123.0' --header 'Accept: text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,*/*;q=0.8' --header 'Accept-Language: en-US,en;q=0.5' --referer 'https://drive.usercontent.google.com/' --header 'Cookie: NID=512=cxpFxcxYHtJXDTmVhW22poOdBMX9hZOmEPPNZxsQdNcz2d2V1IyKxV-lWz-eWx7IsMed5oNhGthbSBJz4wG0iGjnyf1BHWmZ98zOpRSiXCXEBxvnSGoQgmBnMl1MWvDfSB4k_XpFtIQFXPqhQ_EmiGufwZ458QAccgfwI_O3UU8tNwUSD6Cm3e1rvUMGnjHLV9CvgqAsBg; AEC=Ae3NU9NQkR2a7Ndqa_1t-Ywer6Dn3nE8ksvX3Z5_w5yqQ5yZWkhFKehM7Q; 1P_JAR=2024-03-06-06' --header 'Upgrade-Insecure-Requests: 1' --header 'Sec-Fetch-Dest: document' --header 'Sec-Fetch-Mode: navigate' --header 'Sec-Fetch-Site: cross-site' --header 'Sec-Fetch-User: ?1' 'https://drive.usercontent.google.com/download?id=13yTfGRqoVKf-LONaus_QNTJAsgAcw1GU&export=download&confirm=t&uuid=3a7fbafc-5fd4-44f8-bcb2-9b1421e8e7ca' --output-document 'recipes.db'

--2024-03-07 05:04:35--  https://drive.usercontent.google.com/download?id=13yTfGRqoVKf-LONaus_QNTJAsgAcw1GU&export=download&confirm=t&uuid=3a7fbafc-5fd4-44f8-bcb2-9b1421e8e7ca
Resolving drive.usercontent.google.com (drive.usercontent.google.com)... 172.253.117.132, 2607:f8b0:400e:c05::84
Connecting to drive.usercontent.google.com (drive.usercontent.google.com)|172.253.117.132|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 382431232 (365M) [application/octet-stream]
Saving to: ‘recipes.db’

recipes.db          100%[===================>] 364.71M  86.4MB/s    in 4.5s    

2024-03-07 05:04:40 (81.5 MB/s) - ‘recipes.db’ saved [382431232/382431232]



###convert Sqlite database to .CSV format
####save food'image into other directory for image retrieval task

In [4]:
from PIL import Image
import io
import sqlite3
import pandas as pd

conn = sqlite3.connect('/content/recipes.db')
clients = pd.read_sql('SELECT * FROM recipes', conn)
clients.to_csv('csvdata.csv', index=False, encoding='utf-8-sig')


conn = sqlite3.connect('/content/recipes.db')
cursor = conn.cursor()
cursor.execute("SELECT id, img1 FROM recipes")
rows = cursor.fetchall()

for row in rows:
    img_id = row[0]
    img_data = row[1]
    img = Image.open(io.BytesIO(img_data))
    img.save(f"/content/images/{img_id}.png")
conn.close()

In [117]:
data = pd.read_csv('csvdata.csv')
df2 = data.head(25)
df2.drop('img1', axis=1,inplace=True)

<ipython-input-117-84e1272eedc5>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df2.drop('img1', axis=1,inplace=True)


In [160]:

def split_and_unique(categories):
    # Split categories based on commas and remove leading/trailing spaces
    categories_list = [category.strip() for category in categories.split(',')]
    # Remove duplicates and sort the list
    unique_categories = sorted(set(categories_list))
    # Join the unique categories back into a string separated by commas
    return ','.join(unique_categories)

# Apply the function to the 'categories' column and overwrite the original values
df2['categories'] = df2['categories'].apply(split_and_unique)


<ipython-input-160-3b154f1a2268>:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df2['categories'] = df2['categories'].apply(split_and_unique)


### define function to apply some preprocessing operation on dataset

In [161]:
from hazm import Normalizer, word_tokenize, stopwords_list
import string
from gensim.models import Word2Vec

def preprocess_text(text):
    # Normalize the text (remove diacritics, etc.)
    normalizer = Normalizer()
    normalized_text = normalizer.normalize(text)

    # Remove punctuation
    punctuation_chars = string.punctuation + '،؛؟«»،٪ء'
    normalized_text = normalized_text.translate(str.maketrans('', '', punctuation_chars))

    # Convert to lowercase
    normalized_text = normalized_text.lower()

    # Remove stopwords
    stopwords = stopwords_list()
    words = word_tokenize(normalized_text)
    filtered_words = [word for word in words if word not in stopwords]

    return filtered_words



In [162]:
#Preprocess the text
df2['title'] = df2['title'].apply(preprocess_text)
df2['ingredients'] = df2['ingredients'].apply(preprocess_text)
df2['instructions'] = df2['instructions'].apply(preprocess_text)

word2vec_model = Word2Vec(sentences=df2['title'] + df2['ingredients'] + df2['instructions'] , vector_size=100, window=5, min_count=1, workers=4)


<ipython-input-162-1fcd14a4efb6>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df2['title'] = df2['title'].apply(preprocess_text)
<ipython-input-162-1fcd14a4efb6>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df2['ingredients'] = df2['ingredients'].apply(preprocess_text)
<ipython-input-162-1fcd14a4efb6>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pan

### Vectorizing foods and find similar foods vector

In [163]:
from hazm import Normalizer, word_tokenize, stopwords_list
import string
from gensim.models import Word2Vec
from sklearn.metrics import jaccard_score
import numpy as np


# Calculate similarity between food descriptions using Word2Vec model
similarities = []
for idx, row in df2.iterrows():
    title_similar_words = word2vec_model.wv.most_similar(row['title'])
    ingredients_similar_words = word2vec_model.wv.most_similar(row['ingredients'])
    instructions_similar_words = word2vec_model.wv.most_similar(row['instructions'])


    title_similar_words = set([word for word, _ in title_similar_words])
    ingredients_similar_words = set([word for word, _ in ingredients_similar_words])
    instructions_similar_words = set([word for word, _ in instructions_similar_words])


    intersection_cnt = len(title_similar_words.intersection(ingredients_similar_words).intersection(instructions_similar_words))
    union_cnt = len(title_similar_words.union(ingredients_similar_words).union(instructions_similar_words))

    similarity = intersection_cnt / union_cnt if union_cnt != 0 else 0
    similarities.append(similarity)

specific_food_index = 3
similarities_array = np.array(similarities)
top_related_food_indices = similarities_array.argsort()[-6:-1][::-1]  # Exclude the food itself
top_related_food_indices = top_related_food_indices.tolist()

# Print top 5 related foods
top_related_foods = df2.iloc[top_related_food_indices]['title']
print(top_related_foods)


21    [هویج, پلو, تن, ماهی]
14          [ته, چین, میگو]
13     [ته, چین, قارچ, مرغ]
4            [ته, چین, لبو]
6           [ته, چین, گوشت]
Name: title, dtype: object


###**پشنهاد غذا بر اساس مواد اولیه آنها **

###**کاربر می تواند با وارد کردن مواد اولیه اش یک دستور پخت برای غذا دریافت کند **






In [ ]:
del df2
df2 = data.head(25)
df2.drop('img1', axis=1,inplace=True)

!wget --header 'Host: drive.usercontent.google.com' --user-agent 'Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:123.0) Gecko/20100101 Firefox/123.0' --header 'Accept: text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,*/*;q=0.8' --header 'Accept-Language: en-US,en;q=0.5' --referer 'https://drive.google.com/' --header 'Cookie: NID=512=cxpFxcxYHtJXDTmVhW22poOdBMX9hZOmEPPNZxsQdNcz2d2V1IyKxV-lWz-eWx7IsMed5oNhGthbSBJz4wG0iGjnyf1BHWmZ98zOpRSiXCXEBxvnSGoQgmBnMl1MWvDfSB4k_XpFtIQFXPqhQ_EmiGufwZ458QAccgfwI_O3UU8tNwUSD6Cm3e1rvUMGnjHLV9CvgqAsBg; AEC=Ae3NU9NQkR2a7Ndqa_1t-Ywer6Dn3nE8ksvX3Z5_w5yqQ5yZWkhFKehM7Q; 1P_JAR=2024-03-06-06' --header 'Upgrade-Insecure-Requests: 1' --header 'Sec-Fetch-Dest: document' --header 'Sec-Fetch-Mode: navigate' --header 'Sec-Fetch-Site: same-site' 'https://drive.usercontent.google.com/download?id=1Q3JK4NVUC2t5QT63aDiVrCRBV225E_B3&export=download' --output-document 'pos_tagger.model'
!pip install hazm

import numpy as np
import nltk
import re
import string
import warnings
import gensim
from sklearn.metrics.pairwise import cosine_similarity
from configparser import ConfigParser
from functools import reduce
from gensim.models import Doc2Vec
from hazm import Normalizer
import string
from hazm import *

In [141]:
df2 = data.head(25)
keyword_count = 50
stopwords = ['تعداد','دانه' , 'لیتر',  'مقدار'  , 'اندازه لازم' ,'خرد', 'متوسط' , 'به' ,'عدد',  'استکان' ,'نفر', 'پیمانه','قالب','ورق','رنده','کیلو', 'لیوان', 'میزان دلخواه' , 'میزان' , 'قاشق',  'در', 'برای', 'با', 'به', 'از', 'و', 'را', 'که', 'این', 'آن', 'یا', 'هم', 'تا', 'هر','یک' 'دو','سه','چهار','پنج','شش',',پیمانه','ق غ','عدد','قاشق چایخوری','حبه','گرم', 'عدد','بزرگ','کوچک','میزان دلخواه']
model_path = '/content/pos_tagger.model'
tagger = POSTagger(model = model_path)

### Extracting ingredient collection's name based on the given grammar

In [142]:
grammers = [
"""
NP:
        {<NOUN,EZ>?<NOUN.*>}    # Noun(s) + Noun(optional)
"""
]

def extract_candidates(tagged, grammer):
    keyphrase_candidate = set()
    np_parser = nltk.RegexpParser(grammer)
    trees = np_parser.parse_sents(tagged)
    for tree in trees:
        for subtree in tree.subtrees(filter=lambda t: t.label() == 'NP'):
            keyphrase_candidate.add(' '.join(word for word, tag in subtree.leaves()))
    keyphrase_candidate = {kp for kp in keyphrase_candidate if len(kp.split()) <= 25}
    keyphrase_candidate = list(keyphrase_candidate)
    return keyphrase_candidate

### Delete stopwords and apply other preprocessing operation on ingredients list

In [143]:
def replace_stopwords(token, stoplist):
    words = token.split()
    cleaned_words = [word if word not in stoplist else '' for word in words]
    cleaned_token = ' '.join(cleaned_words)
    return cleaned_token


def preprocess_tokens(tokens, stoplist):
    normalizer = Normalizer()
    cleaned_tokens = [token.replace(',', '').replace('\u200c', '').replace('=', '') for token in tokens if token not in string.punctuation and token != '\u200c']
    cleaned_tokens = [normalizer.normalize(token) for token in cleaned_tokens]
    cleaned_tokens = [token for token in cleaned_tokens if token not in stoplist]
    cleaned_tokens = [replace_stopwords(token, stoplist) for token in cleaned_tokens]
    cleaned_tokens = [token.strip() for token in cleaned_tokens if token not in stoplist and len(token) >= 3]
    return cleaned_tokens

In [144]:
for index, row in df2.iterrows():
    text = row['ingredients']

    normalizer = Normalizer()
    normalize_text = normalizer.normalize(text)
    tokenize_text = []
    for sentence in sent_tokenize(normalize_text):
      words = word_tokenize(sentence)
      words = [word for word in words if word not in stopwords]
      tokenize_text.append(words)

    token_tag_list = tagger.tag_sents(tokenize_text)


    for grammer in grammers:
      all_candidates = set()
      all_candidates.update(extract_candidates(token_tag_list, grammer))
      all_candidates = np.array(list(all_candidates))

    cleaned_tokens = preprocess_tokens(all_candidates, stopwords)
    df2.at[index, 'cleaned_ingredients'] = str(set(cleaned_tokens))
    print(index , " : " , cleaned_tokens)

<ipython-input-144-97f09ca7d09d>:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df2.at[index, 'cleaned_ingredients'] = str(set(cleaned_tokens))


0  :  ['کشمش', 'گوشت گوسفند', 'صورت تمایل', 'گوساله مرغ', ' روغن', 'پلویی', 'قیسی زردآلو', 'گردو', 'پیازداغ', 'همراهی برنج', 'برنج', 'فلفل']
1  :  ['غ گوشت', 'کشمش', ' آب', 'بلغورگندم', 'پلویی', 'سیبزمینی', 'زردچوبه', 'روغن', 'قیمه', 'طلایی', 'رب گوجهفرنگی', 'پیاز', ' نخود', 'هویج', 'نمک', 'فلفل']
2  :  ['کشمش', 'دسرها', 'چلو', 'کرده', 'قرار', 'ادویه', 'تزیین', 'مرغ', 'مناسبند فریزر', 'گرد', 'زرشک', 'هسته', 'بادام', 'ترد', 'لواش', 'صورت تمایل', 'آلو', 'کته سفره', 'بیدانه لواش', 'برنج', 'مثلثی', ' بادام', 'مورد برنج', 'اسفناج', 'دادن فر', 'سینه', 'پیاز', 'پوست', 'نمک فلفل', 'تفت', 'کره روغن', 'نکات', 'قالبها', 'بخارا', 'خلال', 'حیوانی زعفران', 'یوفکا', 'خونه']
3  :  ['تزیین', 'کشمش', 'گوشت', ' زعفران', 'غذا', 'خوری', 'گرد لیمو', 'روغن', 'برنج', 'پیاز', 'فلفل', ' نمک']
4  :  ['فیله ', 'تخممرغ', 'آب لبو', ' مرغ', 'لبو', ' روغن', 'وکره', 'پر رتگ', 'پیاز پیاز', 'زعفران', 'برنج', 'ریشه']
5  :  [' ماست', 'دلخواه نمک', 'مرزه', 'زرشک', 'زردچوبه', 'تخممرغ', 'ترخان', 'برنج', 'غذاخوری', 'زعفران', 

### find top 5 related foods based on ingredients of foods
### here we ar searching for a food based on it's ID

In [148]:
def retrieve_similar_foods(food_id, df2):
    input_food_ingredients = set(df2[df2['id'] == food_id]['cleaned_ingredients'].iloc[0])
    # Calculate the intersection between the cleaned ingredients of the given food and all other foods
    df2['intersection_count'] = df2['cleaned_ingredients'].apply(lambda x: len(input_food_ingredients.intersection(set(x))))
    # Sort foods by intersection count and retrieve the top 5 related foods
    similar_foods = df2[df2['id']-1 != food_id].nlargest(5, 'intersection_count')[['id', 'title', 'cleaned_ingredients']]
    return similar_foods

food_id = 2
similar_foods = retrieve_similar_foods(food_id, df2)

# Show the food names related to the top 5 related foods
print("--> related foods for " + df2['title'][food_id] + " are : ")
print(similar_foods[['id', 'title']])

user_in = input("please enter ingredients you have to search suitable food : ")
retrieve_similar_foods(food_id, df2):

--> related foods for پلو پرده ای اسفناج با مرغ are : 
    id              title
1    2  دمی بلغور با قیمه
16  17  آلبالو پلو با مرغ
15  16      ته چین آلبالو
17  18         ته چین مرغ
22  23           مرصع پلو


<ipython-input-148-2419004a17bc>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df2['intersection_count'] = df2['cleaned_ingredients'].apply(lambda x: len(input_food_ingredients.intersection(set(x))))


####user can enter his/her ingredient list and get some recommendation from system

In [155]:
def retrieve_similar_foods(entered_ingredients, df2):
    entered_ingredients = set(preprocess_tokens(entered_ingredients, stopwords))

    # Calculate the intersection between the entered ingredients and the cleaned ingredients of all foods
    df2['intersection_count'] = df2['cleaned_ingredients'].apply(lambda x: len(entered_ingredients.intersection(set(x))))

    # Sort foods by intersection count and retrieve the top 5 related foods
    similar_foods = df2.nlargest(5, 'intersection_count')[['id', 'title', 'cleaned_ingredients']]

    return similar_foods

entered_ingredients = ['لازم نمک', ' روغن', 'کره', 'لازم شیر', 'برنج']
similar_foods_by_ingredients = retrieve_similar_foods(entered_ingredients, df2)
print("--> Suitable foods based on the provided ingredients are : ")
print(similar_foods_by_ingredients[['id', 'title']])


--> Suitable foods based on the provided ingredients are : 
   id                      title
0   1            پلو مطنجنه هندی
1   2          دمی بلغور با قیمه
2   3  پلو پرده ای اسفناج با مرغ
3   4      ته چین گوشت (گرمساری)
4   5                 ته چین لبو


<ipython-input-155-d5313a528b3e>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df2['intersection_count'] = df2['cleaned_ingredients'].apply(lambda x: len(entered_ingredients.intersection(set(x))))
